In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import itertools
import math

%matplotlib inline

# pd.set_option("display.max_rows", 10)

np.random.seed(2)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.wrappers.scikit_learn import KerasRegressor


In [11]:
# Load the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train_X = train.iloc[:,1:]
train_y = train.iloc[:,-1]

train_X.shape

(1460, 80)

In [12]:
set_1_num = train_X[['OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'SalePrice']]
set_1_cat = train_X[['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'SaleCondition', 'SaleType']]
set_1_cat_test = test[['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'SaleCondition', 'SaleType']]

set_2 = train_X[['BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'SalePrice']]

set_3 = train_X[['GrLivArea', 'LotArea', 'GarageArea', 'OpenPorchSF', 'SalePrice']]

set_1_cat[set_1_cat['Electrical'] == np.nan]

set_1_cat['Electrical'] = set_1_cat['Electrical'].fillna('Unk')


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:

labels = np.array([])
for c_name in set_1_cat.columns.values:
    labels = np.append(labels, set_1_cat[c_name].unique())
for c_name in set_1_cat_test.columns.values:
    labels = np.append(labels, set_1_cat_test[c_name].unique())
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(labels.tolist())

for c_name in set_1_cat.columns.values:
    set_1_cat[c_name] = label_encoder.transform(set_1_cat[c_name])
    
train_X = pd.concat([set_1_num, set_1_cat, set_2, set_3], axis=1)

train_X = train_X.drop('SalePrice', axis=1)

print(train_X.shape)

train_X.sample(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


(1460, 32)


,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MSZoning,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,Exterior1st,Exterior2nd,ExterCond,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,SaleCondition,SaleType,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,GrLivArea,LotArea,GarageArea,OpenPorchSF
503,7,8,1959,1997,88,10,62,35,75,75,2,3,51,20,114,105,53,105,117,94,105,76,112,1,2,0,1,1,1801,15602,484,54
101,6,5,1985,1985,88,10,62,99,75,75,2,6,51,58,58,105,53,105,117,94,55,76,112,0,2,1,3,1,1732,9206,476,46
608,8,6,1934,1998,88,10,62,35,75,75,2,6,51,20,114,105,53,105,117,94,105,11,112,0,2,1,4,1,3194,12168,380,0
1089,8,5,2005,2005,42,10,62,101,75,75,108,3,51,65,65,105,53,39,117,94,55,76,112,1,1,1,1,1,1247,3316,550,84
819,7,5,2009,2010,88,10,62,77,75,75,108,3,51,111,111,105,53,39,117,94,55,82,72,1,2,0,2,1,1358,6371,484,35


In [37]:
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
reg = LinearRegression().fit(train_X, train_y)

predict = reg.predict(train_X).round()

print('RMSE = %.6f' % math.sqrt(mean_squared_error(predict, train_y)))


RMSE = 35133.027430


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
set_1_num = test[['OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd']]
set_1_cat = test[['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'SaleCondition', 'SaleType']]

set_2 = test[['BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', ]]

set_3 = test[['GrLivArea', 'LotArea', 'GarageArea', 'OpenPorchSF', ]]

set_1_cat['Electrical'] = set_1_cat['Electrical'].fillna('Unk')
set_1_cat['Utilities'] = set_1_cat['Utilities'].fillna('Unk')
set_1_cat['MSZoning'] = set_1_cat['MSZoning'].fillna('Unk')
set_1_cat['Exterior1st'] = set_1_cat['Exterior1st'].fillna('Unk')
set_1_cat['Exterior2nd'] = set_1_cat['Exterior2nd'].fillna('Unk')
set_1_cat['KitchenQual'] = set_1_cat['KitchenQual'].fillna('Unk')
set_1_cat['SaleType'] = set_1_cat['SaleType'].fillna('Unk')
set_2['BsmtFullBath'] = set_2['BsmtFullBath'].fillna(0)
set_3['GarageArea'] = set_3['GarageArea'].fillna(0)

labels = np.array([])

for c_name in set_1_cat.columns.values:
    set_1_cat[c_name] = label_encoder.transform(set_1_cat[c_name])
    
test_p = pd.concat([set_1_num, set_1_cat, set_2, set_3], axis=1)

test_predict = reg.predict(test_p).round()

test_predict = pd.DataFrame(test_predict)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [20]:


test_predict['Id'] = test['Id']

test_predict.columns = ['SalePrice', 'Id']

test_predict.to_csv('submission.csv', index=False)
